In [1]:
import argparse
import time
from openai import OpenAI, OpenAIError
from api_key import API_KEY
import random
import pandas as pd

In [2]:
def query_openai_model(api_key: str, prompt: str, model: str = "gpt-4o", n: int = 1, max_tokens: int = 100, temperature: float = 0):
    client = OpenAI(api_key=api_key)

    completions = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        n=n,
        stop=None,
        temperature=temperature,
    )
    outputs = []
    for choice in completions.choices:
        outputs.append(choice.message.content.strip())
    return outputs

In [48]:
class Args:
    def __init__(self, from_line=600, to_line=714, temperature=1, prompt_file='prompt_A_f.txt'):
        self.from_line = from_line
        self.to_line = to_line
        self.temperature = temperature
        self.prompt_file = prompt_file

In [49]:
def run(infile, outfile, args):
    api_key = API_KEY

    with open(args.prompt_file, encoding='utf-8') as fprompt, open(outfile, 'w', encoding='utf-8') as fout:
        flag = True
        count = 0
        while flag:
            try:
                ori_prompt = ''.join(fprompt.readlines())  
                
                df = pd.read_csv(infile, encoding='utf-8', header=None)
                lines = df.iloc[args.from_line:args.to_line].apply(lambda row: f"{row.iloc[0]}, {row.iloc[1]}", axis=1).tolist()
                
                for _ in range(5):
                    # disorder the sequence of sentences
                    random.shuffle(lines)
                    prompt = ori_prompt  
                    prompt += '\n概念列表:\n{}'.format('\n'.join(str(line) for line in lines))
                    
                    output = query_openai_model(api_key, prompt, n=1, max_tokens=4000, temperature=args.temperature)  
                    print(output)
                    
                    fout.write(f'{output}\n')
                    fout.flush()
                flag = False
            except OpenAIError as e:
                print("Some error happened here: ", e)
                count += 1
                if count > 10:
                    raise e
                time.sleep(1)

if __name__ == "__main__":
    args = Args()
    run('identities_cn_female.csv', f'chatgpt_f_A_{args.temperature}_{args.from_line}_{args.to_line}.txt', args)


["[614#student (honors)]: +1.0\n[629#surgeon]: +2.5\n[687#virtuous person]: -2.0\n[626#superstar]: +3.5\n[621#subordinate]: 0.0\n[647#teacher (of deaf)]: -1.5\n[642#teacher (head)]: -0.5\n[610#stone cutter]: 0.0\n[670#trickster]: +2.0\n[637#taxi driver]: +1.0\n[609#stockholder]: 0.0\n[711#younger brother]: +0.5\n[624#sumo grand champion]: +2.5\n[606#stepfather]: -1.0\n[674#unemployed person]: -2.5\n[634#systems engineer]: -0.5\n[644#teacher (kindergarten)]: -1.0\n[667#transvestite]: +1.5\n[679#university reject]: -2.0\n[676#union member]: 0.0\n[613#student enrollee]: 0.0\n[649#teammate]: +1.5\n[612#student (enrolled)]: 0.0\n[652#textile dyer]: 0.0\n[669#travelling salesman]: +1.0\n[619#student (vocational hs)]: 0.0\n[625#superior]: -0.5\n[658#thug]: +3.0\n[611#student]: 0.0\n[677#university lecturer]: +1.0\n[697#white-collar worker]: 0.0\n[660#tourist]: +2.0\n[666#tramp]: +2.5\n[709#yokel]: 0.0\n[675#union activist]: +1.5\n[673#uncle]: 0.0\n[615#student (jr.college)]: 0.0\n[648#teacher